# Goal

The goal of this Lab is to demonstrate how to create a simple Agent using Pydantic AI.

# LLM Connection

We define the LLM

In [3]:
from pydantic_settings import BaseSettings
import os

class Settings(BaseSettings):
    GOOGLE_API_KEY: str
    MODEL_NAME: str
    CONTEXT7_API_KEY: str
    class Config:
        #ignore extra fields
        extra = "ignore"
        env_file = ".env"

settings = Settings()
os.environ["GOOGLE_API_KEY"] = settings.GOOGLE_API_KEY

# Simple LLM Agent

Simple LLM call

In [4]:
from pydantic_ai import Agent
joke_agent = Agent(settings.MODEL_NAME,
                instructions="You are a comedian, you tell jokes",
)
result = await joke_agent.run("Tell me a joke")
print(result.output)

Alright, alright, settle down folks! I've got one for ya...

Why don't scientists trust atoms?

...Because they make up *everything*!

*ba-dum-tss!* I'll be here all week! Try the veal!


## Structured LLM Call

We define the Pydantic model for the structured output

In [5]:
from pydantic_ai import Agent
from pydantic import BaseModel

class joke_output(BaseModel):
    joke: str

joke_agent = Agent(settings.MODEL_NAME,
                instructions="You are a comedian, you tell jokes",
                output_type=joke_output
)
result = await joke_agent.run("Tell me a joke")
print(result.output)

joke="Why don't scientists trust atoms? Because they make up everything!"


what if we want 4 jokes?

In [6]:
jokes_history = []
for i in range(4):
    result = await joke_agent.run("Tell me a joke")
    jokes_history.append(result.output.joke)

jokes_history

["Why didn't the skeleton cross the road? Because he had no guts!",
 "Why don't scientists trust atoms? Because they make up everything!",
 "Why don't scientists trust atoms? Because they make up everything!",
 "Why don't scientists trust atoms? Because they make up everything!"]

## Dependencies

Add the JokeDependencies to keep track of the jokes

In [7]:
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from dataclasses import dataclass

# We use dataclass because it allows us to add any object as a dependency, in this case a list of strings
@dataclass
class JokeDependencies:
    previous_jokes: list[str]

# Output defined by Pydantic
class joke_output(BaseModel):
    joke: str

joke_agent = Agent(settings.MODEL_NAME,
                instructions="You are a comedian. You are given a list of previous jokes. You need to tell a new joke that is not in the list",
                deps_type=JokeDependencies,
                output_type=joke_output)

@joke_agent.system_prompt  
async def add_previous_jokes(ctx: RunContext[JokeDependencies]) -> str:
    return f"The previous jokes are {ctx.deps.previous_jokes}"

result = await joke_agent.run("Tell me a joke", deps=JokeDependencies(previous_jokes=[]))
print(result.output)

joke="Why don't scientists trust atoms? Because they make up everything!"


Now, what if we want 4 jokes?


In [8]:
jokes_history = JokeDependencies(previous_jokes=[])
for i in range(4):
    result = await joke_agent.run("Tell me a joke", deps=jokes_history)
    jokes_history.previous_jokes.append(result.output.joke)

jokes_history.previous_jokes
    

["Why don't scientists trust atoms? Because they make up everything!",
 'I told my wife she was drawing her eyebrows too high. She looked surprised.',
 "Parallel lines have so much in common. It's a shame they'll never meet.",
 'What do you call a fake noodle? An impasta.']

# Agent with Tools

Definition of tools

In [9]:
from pydantic_settings import BaseSettings
from random import randint
from pydantic_ai import Agent

# We use dataclass because it allows us to add any object as a dependency, in this case a list of strings
@dataclass
class JokeDependencies:
    previous_jokes: list[str]

# Output defined by Pydantic
class joke_output(BaseModel):
    joke: str

joke_agent = Agent(settings.MODEL_NAME,
                instructions="You are a comedian. You are given a list of previous jokes. You need to tell a new joke that is not in the list",
                deps_type=JokeDependencies,
                output_type=joke_output)

@joke_agent.system_prompt  
async def add_previous_joke(ctx: RunContext[JokeDependencies]) -> str:
    return f"The previous jokes are {ctx.deps.previous_jokes}"

async def tell_a_joke(previous_jokes: list[str]=[]) -> str:
    """Creates a new joke, provide a list of previous jokes, if there are no previous jokes, just provide an empty list"""
    result = await joke_agent.run("Tell me a joke", deps=JokeDependencies(previous_jokes=previous_jokes))
    return result.output.joke

def random_number(min_val: int = 1, max_val: int = 100) -> int:
    """Generates a random number in a given range."""
    return randint(min_val, max_val)

## Agent that uses tools

We define the agent with the new output model and the tools.

In [10]:
class PoemOutput(BaseModel):
    jokes: list[str]
    numbers: list[int]
    poem: str

agent = Agent(settings.MODEL_NAME, tools=[tell_a_joke, random_number], output_type=PoemOutput)

# Run agent
nodes = []
async with agent.iter(
    "Please give me 3 random jokes and 5 random numbers. Then write a small poem about them.",
) as agent_run:
    async for node in agent_run:
        # Each node represents a step in the agent's execution
        print(node)
        nodes.append(node)

UserPromptNode(user_prompt='Please give me 3 random jokes and 5 random numbers. Then write a small poem about them.', instructions=None, instructions_functions=[], system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})
ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Please give me 3 random jokes and 5 random numbers. Then write a small poem about them.', timestamp=datetime.datetime(2025, 9, 3, 19, 21, 55, 358731, tzinfo=datetime.timezone.utc))]))
CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='tell_a_joke', args={'previous_jokes': []}, tool_call_id='pyd_ai_e241f74e0084433c836fa3edf8d7b0e8')], usage=RequestUsage(input_tokens=228, output_tokens=172, details={'thoughts_tokens': 154, 'text_prompt_tokens': 228}), model_name='gemini-2.5-flash', timestamp=datetime.datetime(2025, 9, 3, 19, 21, 57, 118682, tzinfo=datetime.timezone.utc), provider_name='google-gla', provider_details={'finish_reason': 'STOP'}, provider_resp

In [11]:
print(nodes[-1].data.output.model_dump_json(indent=2))

{
  "jokes": [
    "Why don't scientists trust atoms? Because they make up everything!",
    "I told my wife she was drawing her eyebrows too high. She looked surprised.",
    "What do you call a fake noodle? An impasta!"
  ],
  "numbers": [
    85,
    3,
    10,
    27,
    94
  ],
  "poem": "Three jokes were told, with laughter bright,\nOf atoms, eyebrows, and noodle's plight.\nThen numbers appeared, a curious few,\n85, 3, 10, 27, and 94 too.\nA mix of fun, a playful spree,\nFor you and for me, a happy decree."
}


# MCP Agents

## API MCPs: Context7

In [12]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerStreamableHTTP

context7_server = MCPServerStreamableHTTP(
    url ='https://mcp.context7.com/mcp',
    headers={'CONTEXT7_API_KEY': settings.CONTEXT7_API_KEY}
)  

mcp_agent = Agent(
    settings.MODEL_NAME, toolsets=[context7_server], 
    instructions="You are a helpful assistant that can answer questions about code, you use Context7 to search first for the library and then ask some questions to context7 about that library, based on the results, you give an answer to the user"
)  
# Run the Agent
async with mcp_agent:  
    nodes = []
    async with mcp_agent.iter(
        "Give me the code for a simple agent that is connected to an MCP, using ",
    ) as agent_run:
        async for node in agent_run:
            # Each node represents a step in the agent's execution
            print(node)
            nodes.append(node)

UserPromptNode(user_prompt='Give me the code for a simple agent that is connected to an MCP, using ', instructions='You are a helpful assistant that can answer questions about code, you use Context7 to search first for the library and then ask some questions to context7 about that library, based on the results, you give an answer to the user', instructions_functions=[], system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})
ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Give me the code for a simple agent that is connected to an MCP, using ', timestamp=datetime.datetime(2025, 9, 3, 19, 22, 23, 546851, tzinfo=datetime.timezone.utc))], instructions='You are a helpful assistant that can answer questions about code, you use Context7 to search first for the library and then ask some questions to context7 about that library, based on the results, you give an answer to the user'))
CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(

# Display Results

In [13]:
print(nodes[-1].data.output)

```python
import asyncio
import os

from mcp_agent.app import MCPApp
from mcp_agent.agents.agent import Agent
from mcp_agent.workflows.llm.augmented_llm_openai import OpenAIAugmentedLLM

app = MCPApp(name="hello_world_agent")

async def example_usage():
    async with app.run() as mcp_agent_app:
        logger = mcp_agent_app.logger
        # This agent can read the filesystem or fetch URLs
        finder_agent = Agent(
            name="finder",
            instruction="""You can read local files or fetch URLs.
                Return the requested information when asked.""",
            server_names=["fetch", "filesystem"], # MCP servers this Agent can use
        )

        async with finder_agent:
            # Automatically initializes the MCP servers and adds their tools for LLM use
            tools = await finder_agent.list_tools()
            logger.info(f"Tools available:", data=tools)

            # Attach an OpenAI LLM to the agent (defaults to GPT-4o)
            llm = awai

## Command MCPs: DuckDuckGo

In [14]:
from pydantic_ai.mcp import MCPServerStdio

ddg_server = MCPServerStdio(  
    'uvx',
    args=[
        'duckduckgo-mcp-server',
    ]
)
mcp_agent = Agent(settings.MODEL_NAME, toolsets=[ddg_server], 
                  instructions="You are a helpful assistant that can use the DuckDuckGo MCP server to search the web and obtain current information, you always search first and then fetch the content of the search results you think are relevant to the question, finally you answer the question based on the content of the search results")  
# Run the Agent
async with mcp_agent:  
    nodes = []
    async with mcp_agent.iter(
        "What is the weather in Tokyo?",
    ) as agent_run:
        async for node in agent_run:
            # Each node represents a step in the agent's execution
            print(node)
            nodes.append(node)

UserPromptNode(user_prompt='What is the weather in Tokyo?', instructions='You are a helpful assistant that can use the DuckDuckGo MCP server to search the web and obtain current information, you always search first and then fetch the content of the search results you think are relevant to the question, finally you answer the question based on the content of the search results', instructions_functions=[], system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})
ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='What is the weather in Tokyo?', timestamp=datetime.datetime(2025, 9, 3, 19, 22, 46, 811758, tzinfo=datetime.timezone.utc))], instructions='You are a helpful assistant that can use the DuckDuckGo MCP server to search the web and obtain current information, you always search first and then fetch the content of the search results you think are relevant to the question, finally you answer the question based on the content of the search resu

# Display Results

In [15]:
print(nodes[-1].data.output)

The weather in Tokyo, Japan, as of 4:22 AM, is 81°F with light rain and a RealFeel® of 88°. Wind is from the NNE at 6 mph with gusts up to 11 mph. The air quality is poor.

The forecast for today, September 4th, is cloudy, very humid, and not as hot, with occasional rain in the afternoon, reaching a high of 83°. Tonight, there will be a thunderstorm around in the evening, otherwise cloudy and very humid, with a low of 77°. Rain is expected to continue for the next 48 minutes. Thunderstorms, some heavy, are expected on Friday.


## Conclusion

In this notebook, we explored the fundamentals of creating AI agents using Pydantic AI:

### Key Takeaways

1. **Simple Agents**: Created basic agents that can generate text responses with customizable instructions
2. **Structured Output**: Used Pydantic models to ensure agents return well-formatted, typed data instead of plain text
3. **Dependencies**: Implemented stateful agents that can maintain context (like previous jokes) between interactions
4. **Tool Integration**: Built agents that can use external tools (random number generation, joke creation) to perform complex tasks
5. **MCP Integration**: Connected agents to external services through Model Context Protocol (MCP) servers:
   - **API MCPs**: Used Context7 for code documentation and library search
   - **Command MCPs**: Used DuckDuckGo for real-time web search and content fetching

### What We Learned

- Pydantic AI makes it easy to build reliable agents with structured outputs
- Dependencies allow agents to maintain state and context across interactions
- Tools enable agents to perform actions beyond text generation
- MCP servers provide a standardized way to connect agents to external services
- Agents can automatically chain tool calls to complete complex, multi-step tasks

This foundation prepares us for building more sophisticated agent architectures in the following notebooks.
